<a href="https://colab.research.google.com/github/rudrajoshi2481/Bioinformatics-Practices/blob/main/pyNotebooks/Day01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"Hello World Day02"

'Hello World'